In [1]:
import os
import cv2
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

# Mendefinisikan folder input
input_folder = 'Data_Enhance'

# Mendefinisikan daftar parameter GLCM
glcm_parameters = ['contrast', 'homogeneity', 'energy', 'entropy']
directions = [0, 45, 90, 135]

# Membuat list untuk menyimpan data ekstraksi
data = []

# Melakukan iterasi pada setiap file dalam folder input
for filename in tqdm(os.listdir(input_folder), desc="Ekstraksi Gambar"):
    # Mengecek apakah file adalah file gambar
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Membaca gambar menggunakan OpenCV
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path, 0)  # Membaca gambar dalam mode grayscale
        
        # Menghitung matriks GLCM dengan arah 0, 45, 90, 135 derajat
        glcm = graycomatrix(image, distances=[1], angles=directions, levels=256,
                            symmetric=True, normed=True)
        
        # Menghitung nilai parameter GLCM
        glcm_features = []
        for prop in glcm_parameters:
            if prop == 'entropy':
                entropy_values = -np.sum(glcm * np.log2(glcm + 1e-10), axis=(0, 1)).ravel()
                glcm_features.extend(entropy_values)
            else:
                prop_values = graycoprops(glcm, prop).ravel()
                glcm_features.extend(prop_values)
        
        # Menyimpan data ekstraksi dalam list
        data.append(glcm_features + [filename])

# Membuat dataframe dari data ekstraksi
columns = [f'{param}_{dir}' for param in glcm_parameters for dir in directions] + ['filename']
df = pd.DataFrame(data, columns=columns)

# Menampilkan dataframe
print(df)
# Create DataFrame from the collected data


Ekstraksi Gambar: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1201/1201 [00:24<00:00, 48.08it/s]

      contrast_0  contrast_45  contrast_90  contrast_135  homogeneity_0   
0     135.534069   347.452848   336.650794    135.534069       0.238120  \
1      46.697733   230.791161   231.684276     46.697733       0.433912   
2      37.134191   152.484781   150.010789     37.134191       0.384247   
3      45.891728   259.912045   255.878348     45.891728       0.600757   
4      55.980147   622.607719   612.113963     55.980147       0.308909   
...          ...          ...          ...           ...            ...   
1195   10.145221   134.304637   137.359747     10.145221       0.651540   
1196   29.421324   891.732026   881.433594     29.421324       0.359017   
1197   28.146446   480.889760   470.667101     28.146446       0.364593   
1198   34.909865   666.328354   658.898375     34.909865       0.358083   
1199   23.022794   658.007719   651.891927     23.022794       0.386235   

      homogeneity_45  homogeneity_90  homogeneity_135  energy_0  energy_45   
0           0.108318 

In [5]:
scaler = StandardScaler()
numeric_columns = df.columns[:-1]
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

print(df)

      contrast_0  contrast_45  contrast_90  contrast_135  homogeneity_0   
0       1.122507    -0.381743    -0.398937      1.122507      -1.654168  \
1       0.063639    -0.834350    -0.810746      0.063639       0.697962   
2      -0.050351    -1.138151    -1.131171     -0.050351       0.101315   
3       0.054032    -0.721371    -0.715827      0.054032       2.702349   
4       0.174279     0.685763     0.681771      0.174279      -0.803746   
...          ...          ...          ...           ...            ...   
1195   -0.372041    -1.208684    -1.180804     -0.372041       3.312433   
1196   -0.142283     1.729872     1.738376     -0.142283      -0.201781   
1197   -0.157479     0.135946     0.126841     -0.157479      -0.134792   
1198   -0.076864     0.855384     0.865317     -0.076864      -0.213005   
1199   -0.218550     0.823102     0.837829     -0.218550       0.125197   

      homogeneity_45  homogeneity_90  homogeneity_135  energy_0  energy_45   
0           0.010091 

In [6]:
df.to_csv(f'glcm_features_{input_folder}.csv', index=False)

In [7]:
print(df.columns)

Index(['contrast_0', 'contrast_45', 'contrast_90', 'contrast_135',
       'homogeneity_0', 'homogeneity_45', 'homogeneity_90', 'homogeneity_135',
       'energy_0', 'energy_45', 'energy_90', 'energy_135', 'entropy_0',
       'entropy_45', 'entropy_90', 'entropy_135', 'filename'],
      dtype='object')
